# Tensorflow 2: Building a Neural Network

## DESCRIBE THE PROBLEM

##Step 1: Install and load the libraries

Install  tensorflow, if not yet installed, and load all the necessary libraries

In [ ]:
#!pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

In [ ]:
import numpy as np
import datetime

from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

## Step 2: Data Preprocessing

Load the dataset from the keras ibrary

In [ ]:
# Load the Fashion NMIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
print("Length train dataset: ", len(X_train))
print("Length test dataset: ", len(X_test))

We have to analyze the dimensions or shape of the dataset

In [ ]:
print("Train dataset shape: ",X_train.shape)
print("Test dataset shape: ",X_test.shape)

### Normalize images

When working with images, we usually recommend to normalize the dataset in the range [0,1]. You just need to divide each pixel by 255, the max value of a pixel in an image.
This action would help our model to train faster and with greater stability.


In [ ]:
# Normalize train dataset
X_train = X_train / 255.0

# Normalize test dataset
X_test = X_test / 255.0

## Show images

Lets print one image in our dataset

In [ ]:
plt.figure()
plt.imshow(X_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

### Reshape the dataset

We are going to build a fully connected neural network and inputs to this nn must be flatten, then we need to reshape ur dataset

In [ ]:
# Reshape the 28x28 matrix of the images to a flatten 784-vector
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Show the new shape
print("Train dataset shape: ", X_train.shape, "Test dataset shape: ", X_test.shape)

## Step 3: Build a Fully connected neural network

In this experiment, we apply the keras framework to help us to define the network in just a few lines of code.

The first action is to create a sequentioal model where layers of nn are connected sequentially.

In [ ]:
model = tf.keras.models.Sequential()

We are going to build a fully connected neural network and the inputs to this nn must be flatten to act as inputs to the neural network. This mean we need to insert a Flatten layer

In [ ]:
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

Now, we add a Dense layer, fully connected layer, with parameters:

- count of neurons: 128
- activation function: ReLU
- input shape: (784, )

Next, we include a dropout layer to reduce overfitting. It is a regularization technique where in every forward pass randomly some neurons are deactivate.

In [ ]:
# Adding the first dense layer
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
# Adding a dropout layer
model.add(tf.keras.layers.Dropout(0.2))


### Add a second fully connected layer

- units: count of classes to predict
- activation function: 'softmax'

In [ ]:
# Add a second dense layer
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

## Step 4: Compile the model

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy
- Metric: Accuracy

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Show the model structure
model.summary()

## Step 5: Train the model

In this simple demo, we just train for a few epochs

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=5)

## Step 6: Evauate the model

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
# Show result
print("Test accuracy: {}".format(test_accuracy))

## Step 7: Make predictions

Now, it is time to apply the model to the test dataset

In [ ]:
# Predict the test dataset
Y_pred_prob = model.predict(X_test)

Show one prediction to inspect the values returned, the probability of each class

In [ ]:
# Show probabilities
print(Y_pred_prob[0])
# Print the class with highest probability
print(np.argmax(Y_pred_prob[0]))
# Print the true label
print(y_test[0])


In [ ]:
print(Y_pred.shape)

In [ ]:
# Calculate the predicted label for test dataset
y_preds= np.argmax(Y_pred_prob, axis=-1)
print(y_preds.shape)

### Create the confussion matrix

In [ ]:
# Calculate the confussion matrix
cm = confusion_matrix(y_test, y_preds)
print('Confusion Matrix\n')
print(cm)

Plotting the confussion matrix will show efficiently how model works. We define a function to plot the matrix

In [ ]:
# Function to plot the confussion matrix
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.2f}; misclass={:0.2f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
# Create a list with the class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# Plot confussion matrix
plot_confusion_matrix(cm, class_names)

## Step 8: Save the model

to save our model we make a dir and call the save method

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
# Save the model
model.save('saved_model/my_model')

## Restore the model saved

When the model is saved using the save method, we can restore and load the model in a new model just calling the load_model. This procedure

In [ ]:
# Load the saved model
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
# Calculate the predicted label for test dataset
y_preds= np.argmax(model.predict(X_test), axis=-1)
# Calculate the confussion matrix
cm = confusion_matrix(y_test, y_preds)
# Plot confussion matrix
plot_confusion_matrix(cm, class_names)